# Imports

## Preparatory Imports

In [ ]:
import sys
import os

src_path = "../src"
sys.path.insert(0, src_path)

hf_token = os.environ["HUGGINGFACE_TOKEN"]

## Main Imports

In [ ]:
from data import load_data
from client import Client
from tester import Tester
import pickle

In [ ]:
with open("../data/runs/run2.pickle", "rb") as conversation_file:
    conversations = pickle.load(conversation_file)

# Test the Prototype

In [ ]:
data = [load_data(f"../data/processed/train{i}.json") for i in range(1, 4)]
entries = data[0] | data[1]
c = Client(
    "mistralai/Mistral-Nemo-Instruct-2407",
    token=hf_token,
)
t = Tester(c, entries)
t.conversations = conversations
indices = sorted(list(entries.keys()))

In [ ]:
N = len(t.conversations)
print(N)

In [ ]:
indices[N]

In [ ]:
print(t.conversations[indices[N-1]])
print(t.conversations[indices[N]])

In [ ]:
t.run(indices[N:])

In [ ]:
N = 977

In [ ]:
print(entries[N].answers)
print(t.conversations[N].answers)

In [ ]:
print(entries[N-1].answers)
print(t.conversations[N-1].answers)

In [ ]:
del t.conversations[N]

In [ ]:
for i in range(N):
    if i not in t.conversations.keys():
        print(i)

In [ ]:
with open("../data/runs/run2.pickle", "wb") as out:
    pickle.dump(t.conversations, out)